In [ ]:
# backtest.py
from readdata import *
# from strategy import Strategy_MA
from backtest import run_backtest
import backtrader as bt
import pandas as pd
from datetime import datetime

class Strategy_MA(bt.Strategy):
    # 均线策略
    params = (
                ('ma5_period', 5),
                ('ma20_period', 20) 
    )  # 设定全局交易策略参数

    def log(self, txt, dt=None):
        # 日志记录函数
        dt = dt or self.datas[0].datetime.date(0)
        print('date:%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        # 初始化交易指令、买卖价格和手续费
        self.order = None
        self.buyprice = None
        self.buycomm = None 
 
        # 添加移动均线指标，内置了talib模块
        self.ma5 = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.ma5_period)
        self.ma20 = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.ma20_period)
        
    def notify_order(self, order):
        # 处理和打印订单信息
        
        # 买卖订单已提交/已接受 - 无需操作
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        # 检查订单是否已完成
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('买入已执行，%.2f' % order.executed.price) # 记录日志
            elif order.issell():
                self.log('卖出已执行，%.2f' % order.executed.price)
                
            self.bar_executed = len(self)
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('订单已取消/保证金不足/拒绝')
            
        # 记录：没有待处理订单
        self.order = None
        
    def notify_trade(self, trade):
        # 处理和打印交易信息
        if not trade.isclosed:
            return

        self.log(f'本次交易毛利润：{trade.pnl:.2f},扣除交易费用后的净利润：{trade.pnlcomm:.2f}')

        if trade.pnlcomm > 0:  # 如果净收益大于0，就认为这次交易盈利，否则认为这次交易亏损（同时输出交易编号）
            self.log(f'交易获利： {trade.ref}')
        else:
            self.log(f'交易亏损： {trade.ref}')  

    def next(self):
        #主要的循环策略执行部分
        
        # 当前资产总价`
        total_value = self.broker.getvalue()
        
        # 检查是否有待处理订单，如果有就不执行此轮操作
        if self.order:
            return
        
        # 回测最后一天停止交易
        if self.datas[0].datetime.date(0) == end:
            return 
        
        # 这里是九成仓买入卖出的策略
        if self.position.size:  # 检查当前是否已持仓（因为该策略只有持仓和空仓两种状态）
            if self.ma5[0] < self.ma20[0] and self.ma5[-1] > self.ma20[-1]:   # 检查是否满足卖出条件
                self.log("总资产价格：%.2f元" % total_value) 
                print("{:-^50s}".format("Split Line"))
                self.log('卖出创建，%.2f' % self.data.close[0])
                self.close()
        elif self.ma5[0] > self.ma20[0] and self.ma5[-1] < self.ma20[-1]:     # 检查是否满足买入条件(此处是否应当用第二天买入价？)
            order_amount = abs((total_value*0.9/self.datas[0].close[0])//100*100)
            self.log("总资产价格：%.2f元" % total_value)
            print("{:-^50s}".format("Split Line"))
            self.log('买入创建，%.2f' % self.data.close[0])
            self.buy(self.datas[0], size=order_amount)
start_date = '2020-11-20'
end_date = '2022-11-20'
start = '2020-11-20'
end = '2022-11-20'
from datetime import datetime
start_date = '2019-10-20'
end_date = '2022-10-20'
start = datetime.strptime(start_date, '%Y-%m-%d')
end = datetime.strptime(end_date, '%Y-%m-%d') 
startcash = 100000.0
freq='d'
code = "sz.000538"
login_baostock()
rs1 = marketinfo(code, start_date, end_date, frequency=freq)
rs = get_result(rs1)
data = rs[['date', 'open', 'close', 'high', 'low', 'volume']].copy()
numeric_columns = ['open', 'close', 'high', 'low', 'volume']
for col in numeric_columns:
    data[col] = data[col].astype(float)
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
logout_baostock()
cerebro, results = run_backtest(Strategy_MA, data, startcash, start, end)
print(f"初始资金: {startcash}\n回测期间：{start.strftime('%Y%m%d')}:{end.strftime('%Y%m%d')}")
evaluate_results(cerebro, results)

login success!
login respond error_code:0
login respond error_msg:success
